In [2]:
!pip install thefuzz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 28.9 MB/s eta 0:00:00


In [9]:
import numpy as np
import pandas as pd
import os
import json
import sys
import difflib
import requests
import time
import bs4
import os
from thefuzz import process, fuzz
from random import shuffle

In [6]:
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DatastreamingFinalProject"

Mounted at /content/drive
/content/drive/My Drive/IDC_MSc/Year1/DataStreamingAlgorithms/DatastreamingFinalProject


In [4]:
def format_word(word):
  if len(word) > 1:
    last_char = word[-1]
    while not last_char.isalpha():
      word = word[:-1]
      last_char = word[-1]

    first_char = word[0]
    while not first_char.isalpha():
      word = word[1:]
      first_char = word[0]

    word = word.lower()

  return word

In [7]:
words_set = []
for i in range(10):
  candidates_file = f"./Dataset/dataset_candidates_{i}.txt"
  with open(candidates_file, 'r') as f:
    words_list = f.read().split('\n')
    words_set.append({"words_list": words_list})

In [14]:
#some_word_with_error = "appel"
some_sentance_with_error = "I wanna appel pleases or a bannana with a paer and a lemone, \
  or a wartermelon if possibe. I hope i don't have an errof in Londaon and in Jerusealm"
correct_sentence = "I wanna apple please or a banana with a pear and a lemon, \
  or a watermelon if possible. I hope i don't have an error in London and in Jerusalem"
diff_words_to_correct = list(set([format_word(x) for x in correct_sentence.split()]) - set([format_word(x) for x in some_sentance_with_error.split()]))
new_sentance = []
words_set_out = []

for index, words_list in enumerate(words_set):
    time_start_naive = time.time()
    words_set_out.append({})
    words_set_out[index]["matched_words"] = []
    words_set_out[index]["matched_scores"] = []
    words_set_out[index]["words_frequency"] = []
    start = time.time()
    for word in some_sentance_with_error.split():
        word = format_word(word)
        if not word in words_list["words_list"]:
            success_percentage = max((len(word) - 1) / len(word) * 100, 75)
            print(f"Scanning {word}, success_percentage is {success_percentage}")
            print("==========================")
            word_results = [(format_word(x[0]), x[1]) for x in process.extract(word, words_list["words_list"], scorer=fuzz.ratio)]# if x[1] >= success_percentage]
            word_results = list(set(word_results))
            if (len(word_results) > 0 and word_results[0][1] < 100):
                list_fixes = list([x[0] for x in word_results])
                list_scores = list([x[1] for x in word_results])
                list_freq = list([words_list["words_list"].count(x[0]) for x in word_results])
                for idx, fix in enumerate(list_fixes):
                    print(f"Potential fix of {word} is {fix}")
                    print(f"Frequency is {list_freq[idx]}")
                    print(f"Match rate is {list_scores[idx]}")
                    print("==========================")
                words_set_out[index]["matched_words"].append(set(list_fixes) if len(list_fixes) > 1 else list_fixes[0])
                words_set_out[index]["matched_scores"].append(set(list_scores) if len(list_scores) > 1 else list_scores[0])
                words_set_out[index]["words_frequency"].append(set(list_freq) if len(list_freq) > 1 else list_freq[0])
            else:
                words_set_out[index]["matched_words"].append(None)
                words_set_out[index]["matched_scores"].append(None)
                words_set_out[index]["words_frequency"].append(None)
                continue
        else:
            print(f"Word {word} is correct. Skipping")
            words_set_out[index]["matched_words"].append(None)
            words_set_out[index]["matched_scores"].append(None)
            words_set_out[index]["words_frequency"].append(None)

    end = time.time()
    print(f"Total runtime of {(end - start)} seconds")
    words_set_out[index]["time_sec"] = (end - start)
    words_set_out[index]["total_memory_bytes"] = sys.getsizeof(word_results) * 3

Word I is correct. Skipping
Word wanna is correct. Skipping
Scanning appel, success_percentage is 80.0
Potential fix of appel is appeal
Frequency is 9
Match rate is 91
Scanning pleases, success_percentage is 85.71428571428571
Potential fix of pleases is please
Frequency is 1
Match rate is 92
Word or is correct. Skipping
Word a is correct. Skipping
Scanning bannana, success_percentage is 85.71428571428571
Potential fix of bannana is bananas
Frequency is 1
Match rate is 86
Potential fix of bannana is bangunan
Frequency is 0
Match rate is 80
Potential fix of bannana is bandanna
Frequency is 2
Match rate is 80
Potential fix of bannana is lannan
Frequency is 0
Match rate is 77
Word with is correct. Skipping
Word a is correct. Skipping
Scanning paer, success_percentage is 75.0
Potential fix of paer is paper
Frequency is 15
Match rate is 89
Word and is correct. Skipping
Word a is correct. Skipping
Scanning lemone, success_percentage is 83.33333333333334
Potential fix of lemone is lemon
Freque

In [15]:
OUT_DIR = './output/1-dictionary-naive-search'
if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)

In [16]:
for index, words_list in enumerate(words_set_out):
    df_comparisons = pd.DataFrame(data=words_list, index=[format_word(x) for x in correct_sentence.split()])
    df_comparisons = df_comparisons.groupby(df_comparisons.index).first()
    df_comparisons.to_csv(f'{OUT_DIR}/dataset_{index}.csv')

In [17]:
df_comparisons

,matched_words,matched_scores,words_frequency,time_sec,total_memory_bytes
I,None,None,None,1.965285,216
a,None,None,None,1.965285,216
an,None,None,None,1.965285,216
and,None,None,None,1.965285,216
apple,appeal,91,13,1.965285,216
banana,"{gannan, qiannan, bandanna, nana, anna}","{80, 73, 77, 71}","{0, 1}",1.965285,216
don't,None,None,None,1.965285,216
error,"{error, errors}","{80, 73}","{2, 4}",1.965285,216
have,None,None,None,1.965285,216
hope,None,None,None,1.965285,216
